In [ ]:
# ------------------------------------------------------------
# Prereqs:
#   export RELAI_API_KEY="relai-..."        # your RELAI API key
#   export OPENAI_API_KEY="sk-..."          # if your agent/tool uses OpenAI
#   pip install relai                   # relai
#   pip install openinference-instrumentation-openai-agents  # optional automatic tracing for openai agents SDK
#
# Here we demonstrate with a simple summarization agent:
# 1. How to run agents in a simulated environment and collect simulation traces/runs.
# 2. How to annotate the simulation runs on RELAI platform (platform.relai.ai) and create an Annotation Benchmark
# 3. (next on `summarization-agent (simulate->annotate->optimize)-part-2.py`) How to optimize the agent over an annotation benchmark.

!pip install relai
!pip install openinference-instrumentation-openai-agents

In [ ]:
import os

os.environ["RELAI_API_KEY"] = "relai-..."  # or set permanently in your system
os.environ["OPENAI_API_KEY"] = "sk-..."  # or set permanently in your system

In [ ]:
from agents import Agent, Runner
from openinference.instrumentation.openai_agents import OpenAIAgentsInstrumentor

from relai import AsyncRELAI, simulated
from relai.benchmark import RELAIAnnotationBenchmark
from relai.critico import Critico
from relai.critico.evaluate import RELAIAnnotationEvaluator
from relai.data import RELAISample, SimulationTape
from relai.logger import tracer_provider
from relai.maestro import Maestro, params, register_param
from relai.mocker.persona import Persona
from relai.simulator import AsyncSimulator, random_env_generator

# ---- Observability (optional but recommended) -------------------------------
OpenAIAgentsInstrumentor().instrument(tracer_provider=tracer_provider)

In [ ]:
# ============================================================================
# STEP 1 — Decorate inputs/tools that will be simulated
# ============================================================================


@simulated
async def get_user_input():
    msg = input("User: ")
    return msg

In [ ]:
# ============================================================================
# STEP 2 — Your agent core
# (additional) To optimize in STEP 5.4, use `register_param` to define tunable
# parameters and `params` to access them in your agent.
# ============================================================================

register_param(
    "model",
    type="model",
    init_value="gpt-4.1-mini",
    desc="LLM model for the agent",
    allowed=["gpt-4o-mini", "gpt-4.1-mini", "gpt-5-mini"],
)
register_param("prompt", type="prompt", init_value="Summarize the given text.", desc="system prompt for the agent")


async def summarization_agent(msg: str):
    agent = Agent(name="Summarization Agent", instructions=params.prompt, model=params.model)
    response = await Runner.run(agent, msg)
    return response.final_output

In [ ]:
# ============================================================================
# STEP 3 — Wrap agent for simulation traces
# ============================================================================


async def agent_fn(tape: SimulationTape):
    input = await get_user_input()
    print("User:", input)  # Debug print
    tape.agent_inputs["user_text"] = input  # trace inputs for later auditing
    response = await summarization_agent(input)
    return {"response": response}

In [ ]:
# ============================================================================
# STEP 4 — Simulate
# ============================================================================


async def simulate():
    # 4.1 — Set up your simulation environment
    # Bind Personas/MockTools to fully-qualified function names
    env_generator = random_env_generator(
        {
            "__main__.get_user_input": [
                Persona(user_persona="You have a piece of news to summarize. Include that as part of your message."),
                Persona(
                    user_persona="You have a piece of article to summarize. Include that as part of your message."
                ),
            ]
        }
        # Alternatively, set up a Persona Set through RELAI platform (platform.relai.ai) and use the code below:
        # {"__main__.get_user_input": PersonaSet(persona_set_id="your_persona_set_id_here")}
    )

    # 4.2 — SIMULATE
    async with AsyncRELAI() as client:
        simulator = AsyncSimulator(
            client=client,
            agent_fn=agent_fn,
            env_generator=env_generator,
            log_runs=True,
        )

        agent_logs = await simulator.run(num_runs=4)
        print(agent_logs)


await simulate()

In [ ]:
# ============================================================================
# STEP 5 — Annotate
# ============================================================================

# Go to RELAI platform (platform.relai.ai) under ->Results->Runs,
# click on individual runs to:
# 1. view and provide feedback to the simulation runs you just executed.
# 2. create an Annotation Benchmark from these runs for future optimization
#    with the "Add to Benchmark" button at the bottom. (IMPORTANT: Make sure
#    only runs corresponding to the current task are included in the benchmark.

In [ ]:
# ============================================================================
# STEP 6 — Simulate->Evalaute->Optimize with annotation benchmark
# ============================================================================
# 6.1 — Load your annotation benchmark created in STEP 5
benchmark = RELAIAnnotationBenchmark(
    benchmark_id="benchmark ID for your annotation benchmark"
)  # replace with your benchmark ID
for sample in benchmark.samples:
    print(sample)  # inspect loaded benchmark samples

In [ ]:
# 6.2 — Set up a environment generator that customizes simulation configs based
# on the benchmark samples
def custom_env_generator(sample: RELAISample | None = None):
    if not sample:
        return {}
    return {
        "__main__.get_user_input": Persona(
            user_persona=sample.extras["simulation_config"]["__main__.get_user_input"]["user_persona"],
            starting_message=sample.agent_inputs["all_inputs"][
                "user_text"
            ],  # provide the original user input recorded
        )
    }

In [ ]:
# For this notebook example, since the agent code is contained in the notebook,
# we create a source.py file containing the agent code for later optimization of agent structure
from IPython import get_ipython


def get_notebook_code():
    ip = get_ipython()
    cells = ip.user_ns["In"]  # This is a list of all executed input cells as strings
    source = ""

    # For example, print everything except the current cell
    for idx, code in enumerate(cells):
        if code and not code.strip().startswith("get_ipython()") and not "import os" in code:
            source += code + "\n"

    return source


with open("source.py", "w") as f:
    f.write(get_notebook_code())

In [ ]:
async def main():
    async with AsyncRELAI() as client:
        # 6.3 — Set up the simulator with the annotation benchmark and custom env generator
        simulator = AsyncSimulator(
            client=client,
            agent_fn=agent_fn,
            env_generator=custom_env_generator,  # use custom env generator
            benchmark=benchmark,  # use the annotation benchmark for simulation
            log_runs=True,
        )

        # 6.4 — Set up Critico with RELAIAnnotationEvaluator for automatic evaluation of annotation benchmarks
        critico = Critico(client=client)
        critico.add_evaluators(evaluators={RELAIAnnotationEvaluator(client=client): 1})

        # 6.5 — OPTIMIZE with Maestro
        maestro = Maestro(client=client, agent_fn=agent_fn, log_to_platform=True, name="Summarization Agent")
        maestro.add_setup(simulator=simulator, critico=critico)
        # one can use multiple simulator+critico setups with different weights by calling `add_setup` multiple times
        # maestro.add_setup(simulator=simulator, critico=critico, weight = 1)
        # maestro.add_setup(simulator=another_simulator, critico=another_critico, weight = 0.5)

        # 6.5.1 — Optimize agent configurations (the parameters registered earlier in STEP 2)
        # params.load("saved_config.json")  # load previous params if available
        await maestro.optimize_config(
            total_rollouts=10,  # Total number of rollouts to use for optimization.
            batch_size=2,  # Base batch size to use for individual optimization steps. Defaults to 4.
            explore_radius=1,  # A positive integer controlling the aggressiveness of exploration during optimization.
            explore_factor=0.5,  # A float between 0 to 1 controlling the exploration-exploitation trade-off.
            verbose=True,  # If True, additional information will be printed during the optimization step.
        )
        params.save("saved_config.json")  # save optimized params for future usage

        # 6.5.2 — Optimize agent structure (changes that cannot be achieved by setting parameters alone)
        await maestro.optimize_structure(
            total_rollouts=10,  # Total number of rollouts to use for optimization.
            code_paths=["source.py"],  # A list of paths corresponding to code implementations of the agent.
            verbose=True,  # If True, additional information will be printed during the optimization step.
        )


await main()